In [ ]:
import cv2
import os
import zipfile
import numpy as np

# Define the file paths
zip_file_path = 'shoe_boot_sanda_2.zip'
extract_folder_path = 'folder'

# Extract the images from the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder_path)

# Create subdirectories for each shoe type
for shoe_type in ['boots', 'shoes', 'sandals']:
    os.makedirs(os.path.join(extract_folder_path, shoe_type), exist_ok=True)

# Define the classifier to identify the type of shoe
classifier = cv2.CascadeClassifier('haar_cascade.xml')

# Iterate through each image and move it to the corresponding subdirectory
for filename in os.listdir(extract_folder_path):
    file_path = os.path.join(extract_folder_path, filename)
    if os.path.isfile(file_path):
        img = cv2.imread(file_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        shoes = classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        if len(shoes) > 0:
            for (x, y, w, h) in shoes:
                shoe_img = img[y:y+h, x:x+w]
                if w > h:
                    # Boot
                    cv2.imwrite(os.path.join('folder/boots', 'boots', filename), shoe_img)
                else:
                    # Sandal or Shoe
                    hsv = cv2.cvtColor(shoe_img, cv2.COLOR_BGR2HSV)
                    lower_red = np.array([0, 70, 50])
                    upper_red = np.array([10, 255, 255])
                    mask = cv2.inRange(hsv, lower_red, upper_red)
                    red_pixels = cv2.countNonZero(mask)
                    if red_pixels > 100:
                        # Sandal
                        cv2.imwrite(os.path.join(extract_folder_path, 'sandals', filename), shoe_img)
                    else:
                        # Shoe
                        cv2.imwrite(os.path.join(extract_folder_path, 'shoes', filename), shoe_img)